<a href="https://colab.research.google.com/github/Hillascher5/nlp-tweets-sentiment-analysis/blob/main/Bertweet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Try a specific model for the task

In [1]:
# # Needed for Google Colab
# !pip install --quiet evaluate transformers optuna datasets nltk scikit-learn
# !pip install numpy==1.26.4

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

from wordcloud import WordCloud, STOPWORDS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset
from torch.utils.data import DataLoader, TensorDataset
from optuna.pruners import MedianPruner
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import os
import re
import string
import time
import glob
import nltk
import evaluate
import transformers
import torch
import optuna
import wandb
wandb.login()
# API key - 0cbd7fe3cffd71df993b30edb4fa0db94f114413

os.environ["WANDB_PROJECT"] = "tweet-sentiment-classification_split_to_test_maxl_256_bertweet"
os.environ["WANDB_INIT_TIMEOUT"] = "180"

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

env: CUDA_LAUNCH_BLOCKING=1
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


wandb: Currently logged in as: hillas (hillas-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# Load data
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_train.csv', encoding='latin1')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_test.csv', encoding='latin1')

In [4]:
# Merge and shuffle for better stratified splits
df_full = pd.concat([df_train, df_test], ignore_index=True)
df_full = df_full.sample(frac=1.0, random_state=42).reset_index(drop=True)

In [5]:
# Try without pre-processing
is_preprocessed = "no_preprocess"
df_full["clean_text"] = df_full["OriginalTweet"]

In [6]:
# Choose pretrained models
bert_model_name = "vinai/bertweet-base"

sentiment_labels = df_full['Sentiment'].unique()
n_labels = len(sentiment_labels)

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=n_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

In [7]:
# Mapping sentiments to unique numeric IDs
unique_labels = sorted(df_full["Sentiment"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
df_full["label"] = df_full["Sentiment"].map(label2id)

In [8]:
# Stratified split: 70% train, 15% val, 15% test
train_val_df, test_df = train_test_split(df_full, test_size=0.15, stratify=df_full["label"], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, stratify=train_val_df["label"], random_state=42)

# Confirm sizes
print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

Train size: 31466
Val size: 6745
Test size: 6744


In [9]:
train_subset_df, _ = train_test_split(
    train_df[["clean_text", "label"]],
    train_size=2000,
    stratify=train_df["label"],
    random_state=42
)

val_subset_df, _ = train_test_split(
    val_df[["clean_text", "label"]],
    train_size=500,
    stratify=val_df["label"],
    random_state=42
)

In [10]:
# Tokenize function
def tokenize_function_bert(examples):
    return bert_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=128)

In [11]:
# Convert DataFrame to Hugging Face Dataset
hf_subset_train = Dataset.from_pandas(train_subset_df)
hf_subset_val = Dataset.from_pandas(val_subset_df)

hf_train = Dataset.from_pandas(train_df[["clean_text", "label"]])
hf_val = Dataset.from_pandas(val_df[["clean_text", "label"]])
hf_test = Dataset.from_pandas(test_df[["clean_text", "label"]])

In [12]:
# Tokenize subsets
# Tokenize for BERT
tokenized_bert_train_sub = hf_subset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val_sub = hf_subset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
# Tokenize full dataset
# Tokenize for BERT
tokenized_bert_train = hf_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val = hf_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_test = hf_test.map(tokenize_function_bert, batched=True)
tokenized_bert_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

In [14]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

In [15]:
def build_trainer(model_checkpoint, trial, run_prefix, train_dataset, val_dataset):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    n_samples = len(train_dataset)

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    run_name = f"{run_prefix}-ep{num_epochs}-lr{learning_rate}-bs{batch_size}-samples{n_samples}-run{int(time.time())}-{is_preprocessed}"

    args = TrainingArguments(
        output_dir=f"./results/{run_prefix}/{run_name}",
        disable_tqdm=True,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        load_best_model_at_end=True,
        save_total_limit=1,
        logging_strategy="epoch",
        logging_dir=f"./logs/{run_prefix}/{run_name}",
        run_name=run_name,
        report_to="wandb",
        metric_for_best_model="f1_macro",
        greater_is_better=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [16]:
def objective_bert(trial):
    trainer = build_trainer(
        model_checkpoint="vinai/bertweet-base",
        trial=trial,
        run_prefix="bertweet",
        train_dataset=tokenized_bert_train_sub,
        val_dataset=tokenized_bert_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

In [17]:
# !huggingface-cli login

In [18]:
study_bert = optuna.create_study(direction="maximize",
                                 pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                 study_name=f"bertweet_study_stratify_{is_preprocessed}",
                                 storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/bertweet_study_stratify_maxl_256_{is_preprocessed}.db",
                                 load_if_exists=True)
study_bert.optimize(objective_bert, n_trials=5)
wandb.finish()

[I 2025-08-07 19:54:10,123] Using an existing study with name 'bertweet_study_stratify_no_preprocess' instead of creating a new one.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/tmp/ipython-input-1416991230.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.5316, 'grad_norm': 2.997807741165161, 'learning_rate': 2.526856073216729e-05, 'epoch': 1.0}
{'eval_loss': 1.414493203163147, 'eval_accuracy': 0.348, 'eval_f1_macro': 0.28976655222285097, 'eval_runtime': 0.4982, 'eval_samples_per_second': 1003.662, 'eval_steps_per_second': 32.117, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.326, 'grad_norm': 5.873935699462891, 'learning_rate': 1.1485709423712405e-06, 'epoch': 2.0}
{'eval_loss': 1.3064384460449219, 'eval_accuracy': 0.424, 'eval_f1_macro': 0.40559039267341, 'eval_runtime': 0.47, 'eval_samples_per_second': 1063.917, 'eval_steps_per_second': 34.045, 'epoch': 2.0}
{'train_runtime': 21.5282, 'train_samples_per_second': 185.803, 'train_steps_per_second': 5.853, 'train_loss': 1.4287896534753224, 'epoch': 2.0}


...

{'loss': 1.2169, 'grad_norm': 9.355369567871094, 'learning_rate': 6.532036951610166e-08, 'epoch': 4.0}
{'eval_loss': 1.3122080564498901, 'eval_accuracy': 0.406, 'eval_f1_macro': 0.4090522053541978, 'eval_runtime': 0.8023, 'eval_samples_per_second': 623.242, 'eval_steps_per_second': 39.888, 'epoch': 4.0}
{'train_runtime': 57.5517, 'train_samples_per_second': 139.005, 'train_steps_per_second': 8.688, 'train_loss': 1.3835128784179687, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 1.3122080564498901, 'eval_accuracy': 0.406, 'eval_f1_macro': 0.4090522053541978, 'eval_runtime': 0.8197, 'eval_samples_per_second': 609.978, 'eval_steps_per_second': 39.039, 'epoch': 4.0}


[I 2025-08-07 19:58:06,999] Trial 7 finished with value: 0.4090522053541978 and parameters: {'learning_rate': 1.088672825268361e-05, 'batch_size': 16, 'num_train_epochs': 4}. Best is trial 5 with value: 0.5541241030428423.


eval/accuracy,▃▅▅▁▁▂▂▄▇███▃▅▇▇██▂▃▄▄▄
eval/f1_macro,▄▆▆▁▂▃▃▅▇███▃▅▇███▂▄▅▆▆
eval/loss,▆▄▄█▇▇▇▅▂▁▁▁▆▄▂▂▁▁▇▆▄▄▄
eval/runtime,▂▁▂▁▁▁▂█████▁▁▁▁▁▂█▇▇██
eval/samples_per_second,▇█▇▇██▇▁▁▁▁▁▇████▇▁▁▁▁▁
eval/steps_per_second,▁▃▁▂▂▂▁▇▇▇▇▇▂▃▂▂▃▁███▇▇
train/epoch,▁▁▃▃▃▃▁▃▃▅▅▅▅▁▃▃▅▅▆▆▆▁▁▃▃▅▅▆████▁▃▃▅▅▆▆▆
train/global_step,▁▁▂▂▂▂▁▂▂▃▃▃▃▂▄▄▆▆███▁▁▂▂▃▃▄▅▅▅▅▂▄▄▆▆███
train/grad_norm,▂▃▁▁▂▂▅▅▇▂▃█▅▅▁▃▃▅
train/learning_rate,█▁▃▂▁▇▅▃▁▇▅▄▂▁▃▃▂▁
train/loss,█▆██▇█▅▃▁█▆▄▂▁█▇▅▅


In [19]:
best_trial_bert = study_bert.best_trial
print('Bert best trial on subset:')
print(best_trial_bert.params)

Bert best trial on subset:
{'learning_rate': 2.6383812026939042e-05, 'batch_size': 16, 'num_train_epochs': 4}


In [21]:
best_params_bert = best_trial_bert.params
run_name_bert = f"bertweet_final_stratify_{is_preprocessed}-ep{best_params_bert['num_train_epochs']}-lr{best_params_bert['learning_rate']:.1e}-bs{best_params_bert['batch_size']}"
wandb.init(project="tweet-sentiment-classification_split_to_test_maxl_256_bertweet", name=run_name_bert, reinit=True)

final_trainer_bert = build_trainer(
    model_checkpoint="vinai/bertweet-base",
    trial=best_trial_bert,
    run_prefix=f"bertweet_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_bert_train,
    val_dataset=tokenized_bert_val
)
final_trainer_bert.train()
final_trainer_bert.evaluate(tokenized_bert_test)
wandb.finish()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/tmp/ipython-input-1416991230.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.8532, 'grad_norm': 12.822455406188965, 'learning_rate': 1.9804625550470512e-05, 'epoch': 1.0}
{'eval_loss': 0.6159510016441345, 'eval_accuracy': 0.7697553743513714, 'eval_f1_macro': 0.7780496635343379, 'eval_runtime': 10.3347, 'eval_samples_per_second': 652.656, 'eval_steps_per_second': 40.833, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.4668, 'grad_norm': 11.943473815917969, 'learning_rate': 1.3212025849788998e-05, 'epoch': 2.0}
{'eval_loss': 0.5208954215049744, 'eval_accuracy': 0.8241660489251297, 'eval_f1_macro': 0.8299760795058827, 'eval_runtime': 10.4731, 'eval_samples_per_second': 644.032, 'eval_steps_per_second': 40.294, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.2223, 'grad_norm': 13.517021179199219, 'learning_rate': 2.011983631947563e-08, 'epoch': 4.0}
{'eval_loss': 0.5478225350379944, 'eval_accuracy': 0.8517420311341735, 'eval_f1_macro': 0.8568488531959199, 'eval_runtime': 10.4672, 'eval_samples_per_second': 644.392, 'eval_steps_per_second': 40.316, 'epoch': 4.0}
{'train_runtime': 749.8086, 'train_samples_per_second': 167.862, 'train_steps_per_second': 10.493, 'train_loss': 0.4663595753268779, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.5796532034873962, 'eval_accuracy': 0.849644128113879, 'eval_f1_macro': 0.8539843764336534, 'eval_runtime': 10.3277, 'eval_samples_per_second': 653.001, 'eval_steps_per_second': 40.861, 'epoch': 4.0}


eval/accuracy,▁▆▇██
eval/f1_macro,▁▆▇██
eval/loss,█▃▁▄▆
eval/runtime,▁▃█▃▁
eval/samples_per_second,█▆▁▆█
eval/steps_per_second,█▆▁▆█
train/epoch,▁▁▃▃▆▆████
train/global_step,▁▁▃▃▆▆████
train/grad_norm,▂▁█▂
train/learning_rate,█▆▃▁
train/loss,█▄▂▁


In [25]:
final_trainer_bert.save_model(f"models/w_test_split/bertweet_final_stratify_{is_preprocessed}")
bert_tokenizer.save_pretrained(f"models/w_test_split/bertweet_final_stratify_{is_preprocessed}")
!cp -r models/w_test_split/bertweet_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/bertweet_best_model_stratify_maxl_256_{is_preprocessed}"